In [29]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import path
import xmltodict
from myfunctions import event_separation , image_crop, image_crop_negative, poi, 
from myfunctions import delete_old_extracted_events
from database.extract_yaml import get_dict
import tifffile
from pathlib import Path

In [ ]:
BASE_DIR = r'//lebnas1.epfl.ch/microsc125/deep_events'
SAVING_SCHEME = "ws_0.1"

## Input data from a folder ##

In [ ]:
files_dir = os.path.join(BASE_DIR, 'original_data')
images_dir = '221123_mtStayGold_U2OS_ZEISS_fl'
joined_path = os.path.join(files_dir, images_dir)
size=(2048,2048)
img,input_name,output_name,datacsv,pixel_size={},{},{},{},{}

path = Path(joined_path)
files = list(path.glob("*.ome.tif"))
files.extend(list(path.glob("*.csv")))


for input_file in files:
    input_file = str(input_file.name)
    joined_file_path = os.path.join(files_dir, images_dir, input_file)
    if '.tif' in input_file:
        tif  = tifffile.TiffFile(joined_file_path)
        mdInfoDict = xmltodict.parse(tif.ome_metadata, force_list={'Plane'})
        date, cell_type, dye, bf_fl, index  = input_file.split('_')
        number, ome, tiff= index.split('.')
        ##
        if int(number)==1 or int(number)==2 or int(number)==4 or int(number)==8:
            pixel_size_value=0
            pixel_size[int(number)-1]=1
        else:
            pixel_size_value=1
            pixel_size[int(number)-1]=0.103
        ##
        img[int(number)-1] = Image.open(joined_file_path)
        input_name[int(number)-1]= f'points_{date}_{cell_type}_{bf_fl}_{number}'
        output_name[int(number)-1]= f'image_{date}_{cell_type}_{bf_fl}_{number}'
        print('Loaded image:'+input_file)
    else:
        labels, index, bf_fl  = input_file.split('_')
        datacsv[int(index)-1] = pd.read_csv(joined_file_path)
        print('Loaded csv:'+ input_file)
print('Pixel scaling:',pixel_size)
max_number=int(len(datacsv))   

## Get all Gaussian Points of Interest ##

In [ ]:
for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    ##
    scale_value=float(pixel_size[x])
    print('scaling by',scale_value)
    print(csv['axis-1'][1])
    csv['axis-1']=csv['axis-1']*(1/scale_value)
    print(csv['axis-1'][1])
    csv['axis-2']=csv['axis-2']*(1/scale_value)
    ##
    for i in range(2,7,1):
        sigma=(i,i)
        s=sigma[0]
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        framenum=im.n_frames
        poi(csv,in_name,sigma,size,framenum)

## Cut Images and Gaussian POIs ##

In [ ]:
base_dir= BASE_DIR
training_dir = os.path.join(base_dir, 'training_data')
folder_name=images_dir+'_pos'
filepath= os.path.join(training_dir , folder_name)
tif_files = list(item for item in files if ".tif" in item.name)

folder_dict = get_dict(images_dir)
delete_old_extracted_events(folder_dict, os.path.join(training_dir, images_dir))
event_dict = copy.deepcopy(folder_dict)
event_dict['type'] = "event"
del event_dict['extracted_events']

for x in range(max_number):
    event_dict['original_file'] = os.path.basename(tif_files[x])
    event_dict['event_content'] = 'division'
    
    csv=datacsv[x]
    im=img[x]
    print('file number:',x+1)
    ##
    # scale_value=float(pixel_size[x])
    # print('scaling by',scale_value)
    # print(csv['axis-1'][1])
    # csv['axis-1']=csv['axis-1']*(1/scale_value)
    # print(csv['axis-1'][1])
    # csv['axis-2']=csv['axis-2']*(1/scale_value)
    ##
    list1=event_separation(csv)
    l=len(list1)
    out_name=f'{output_name[x]}'
    image_crop(l,list1, csv, im,0, out_name,filepath, SAVING_SCHEME=SAVING_SCHEME,
               folder_dict=folder_dict, event_dict=event_dict)

    for i in range(2,7,1):
        sigma=(i,i)
        s=sigma[0]
        print('sigma:',s)
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        out_name_g=f'{output_name[x]}_sigma{s}'
        
        gauss=base_dir+f'\{in_name}'
        gauss_image=Image.open(gauss)
        image_crop(l,list1, csv, gauss_image,1, out_name_g,filepath)

## Cut Negative Frames ##

In [ ]:
base_dir= r'C:\Users\roumba\Documents\Software\deep-events'
training_dir= r'C:\Users\roumba\Documents\Software\deep-events\training_data'
folder_name=images_dir+'_neg'
filepath= os.path.join(training_dir , folder_name)
from myfunctions import image_crop_negative

for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    print('file number:',x)
    out_name=f'{output_name[x]}'

    list1=event_separation(csv)
    l=len(list1)
    image_crop_negative(l,list1, csv, im, out_name,filepath)